In [1]:
from neuralzoo.loader.load_data import dataframes_from_cifar
import tensorflow as tf

# NOTE legacy import from local pickles
# dataframe_train, target_train, dataframe_test, target_test = dataframes_from_cifar()
# target_train.head()
data = tf.keras.datasets.cifar10.load_data()
data

  1933312/170498071 ━━━━━━━━━━━━━━━━━━━━ 45:19 16us/step